In [1]:
import numpy as np
import pandas as pd

In [2]:
pd.set_option('display.max_columns', None)

Overview.

In [3]:
matches = pd.read_csv('matches.csv')

In [4]:
len(matches)

16332

In [5]:
matches.head()

,Unnamed: 0,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,datetime,home_standings,away_standings,home_points,away_points,home_points_last_4_matches,away_points_last_4_matches,home_points_last_2_home_matches,away_points_last_2_away_matches,home_goals_last_4_matches,away_goals_last_4_matches,home_goals_last_2_home_matches,away_goals_last_2_away_matches,home_goals_against_last_4_matches,away_goals_against_last_4_matches,home_goals_against_last_2_home_matches,away_goals_against_last_2_away_matches,home_mean_xg_last_4_matches,away_mean_xg_last_4_matches,home_mean_xg_last_2_home_matches,away_mean_xg_last_2_away_matches,home_mean_xg_against_last_4_matches,away_mean_xg_against_last_4_matches,home_mean_xg_against_last_2_home_matches,away_mean_xg_against_last_2_away_matches
0,9594,Reims,Paris Saint Germain,August 08 2014,Ligue 1,2014/2015,2,2,1.37,2.66,2014-08-08,16,15,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,9596,Evian Thonon Gaillard,Caen,August 09 2014,Ligue 1,2014/2015,0,3,0.81,1.24,2014-08-09,5,4,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9597,Guingamp,Saint-Etienne,August 09 2014,Ligue 1,2014/2015,0,2,0.63,1.50,2014-08-09,5,18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,9598,Lille,Metz,August 09 2014,Ligue 1,2014/2015,0,0,1.54,0.06,2014-08-09,7,11,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,9599,Montpellier,Bordeaux,August 09 2014,Ligue 1,2014/2015,0,1,1.02,0.75,2014-08-09,13,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
ratings = pd.read_csv('ratings.csv')

In [7]:
len(ratings)

640501

In [8]:
ratings.head()

,Unnamed: 0,match_url,player,position,rating,home_team,away_team,season,home_or_away,datetime
0,8560,https://1xbet.whoscored.com/Matches/865957/Liv...,Guglielmo Stendardo,DC,7.54,AC Milan,Atalanta,2014/2015,away,2015-01-18
1,8561,https://1xbet.whoscored.com/Matches/865957/Liv...,Giuseppe Biava,DC,7.91,AC Milan,Atalanta,2014/2015,away,2015-01-18
2,8562,https://1xbet.whoscored.com/Matches/865957/Liv...,Cristiano Del Grosso,DL,7.81,AC Milan,Atalanta,2014/2015,away,2015-01-18
3,8559,https://1xbet.whoscored.com/Matches/865957/Liv...,Yohan Benalouane,DR,7.43,AC Milan,Atalanta,2014/2015,away,2015-01-18
4,8568,https://1xbet.whoscored.com/Matches/865957/Liv...,Mauricio Pinilla,FW,6.49,AC Milan,Atalanta,2014/2015,away,2015-01-18


In [9]:
from datetime import datetime

In [10]:
matches['datetime'] = matches['datetime'].apply(lambda s: datetime.strptime(s, '%Y-%m-%d').date())
ratings['datetime'] = ratings['datetime'].apply(lambda s: datetime.strptime(s, '%Y-%m-%d').date())

In [11]:
matches = matches.sort_values(by=['datetime', 'home_team'])
ratings = ratings.sort_values(by=['datetime', 'home_team', 'home_or_away', 'position', 'rating', 'player'])

Get lineups formation.

In [12]:
sorted(list(ratings['position'].unique()))

['AMC',
 'AML',
 'AMR',
 'DC',
 'DL',
 'DMC',
 'DML',
 'DMR',
 'DR',
 'FW',
 'FWL',
 'FWR',
 'GK',
 'MC',
 'ML',
 'MR',
 'Sub']

In [13]:
def get_formation(DC_cnt, FW_cnt):
    if DC_cnt == 2 and FW_cnt == 1:
        return 0
    if DC_cnt == 3 and FW_cnt == 1:
        return 1
    if DC_cnt == 2 and FW_cnt == 2:
        return 2
    if DC_cnt == 3 and FW_cnt == 2:
        return 3
    return 4

In [14]:
home_formation = []
away_formation = []

prev_teams = ('', '')
positions = ([], [])

for i in range(len(ratings)):
    teams = (ratings.iloc[i]['home_team'], ratings.iloc[i]['away_team'])
    if (teams != prev_teams) and (i >= 36):
        home_DC_cnt = positions[0].count('DC')
        home_FW_cnt = positions[0].count('FW')
        home_formation.append(get_formation(home_DC_cnt, home_FW_cnt))

        away_DC_cnt = positions[1].count('DC')
        away_FW_cnt = positions[1].count('FW')
        away_formation.append(get_formation(away_DC_cnt, away_FW_cnt))

        prev_teams = teams
        positions = ([], [])

    home_flag = (ratings.iloc[i]['home_or_away'] == 'home')
    pos = ratings.iloc[i]['position']
    if home_flag:
        positions[0].append(pos)
    else:
        positions[1].append(pos)

home_DC_cnt = positions[0].count('DC')
home_FW_cnt = positions[0].count('FW')
home_formation.append(get_formation(home_DC_cnt, home_FW_cnt))

away_DC_cnt = positions[1].count('DC')
away_FW_cnt = positions[1].count('FW')
away_formation.append(get_formation(away_DC_cnt, away_FW_cnt))

In [15]:
len(home_formation), len(away_formation)

(16332, 16332)

In [16]:
sorted(set(home_formation)), sorted(set(away_formation))

([0, 1, 2, 3, 4], [0, 1, 2, 3, 4])

In [17]:
matches['home_formation'] = home_formation
matches['away_formation'] = away_formation

In [18]:
home = 'Roma'
away = 'Inter'
season = '2022/2023'

c_1 = (matches['home_team'] == home)
c_2 = (matches['away_team'] == away)
c_3 = (matches['season'] == season)

matches[c_1 & c_2 & c_3]

,Unnamed: 0,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,datetime,home_standings,away_standings,home_points,away_points,home_points_last_4_matches,away_points_last_4_matches,home_points_last_2_home_matches,away_points_last_2_away_matches,home_goals_last_4_matches,away_goals_last_4_matches,home_goals_last_2_home_matches,away_goals_last_2_away_matches,home_goals_against_last_4_matches,away_goals_against_last_4_matches,home_goals_against_last_2_home_matches,away_goals_against_last_2_away_matches,home_mean_xg_last_4_matches,away_mean_xg_last_4_matches,home_mean_xg_last_2_home_matches,away_mean_xg_last_2_away_matches,home_mean_xg_against_last_4_matches,away_mean_xg_against_last_4_matches,home_mean_xg_against_last_2_home_matches,away_mean_xg_against_last_2_away_matches,home_formation,away_formation
16118,16284,Roma,Inter,May 06 2023,Serie A,2022/2023,0,2,0.88,1.22,2023-05-06,7,5,58,60,5.0,9.0,4.0,6.0,6.0,12.0,4.0,9.0,5.0,2.0,1.0,0.0,1.605,2.265,1.8,2.17,1.015,0.7775,1.25,0.645,1,3


In [ ]:
c_1 = (ratings['home_team'] == home)
c_2 = (ratings['away_team'] == away)
c_3 = (ratings['season'] == season)

ratings[c_1 & c_2 & c_3]

Get mean ratings for defense, midfield, attack players.

In [20]:
home_defense_rating = []
away_defense_rating = []
home_midfield_rating = []
away_midfield_rating = []
home_attack_rating = []
away_attack_rating = []

prev_teams = ('', '')
rates = (([], [], []), ([], [], []))

for i in range(len(ratings)):
    teams = (ratings.iloc[i]['home_team'], ratings.iloc[i]['away_team'])
    if (teams != prev_teams) and (i >= 36):
        home_defense_rating.append(np.nanmean(rates[0][0]))
        away_defense_rating.append(np.nanmean(rates[1][0]))
        home_midfield_rating.append(np.nanmean(rates[0][1]))
        away_midfield_rating.append(np.nanmean(rates[1][1]))
        home_attack_rating.append(np.nanmean(rates[0][2]))
        away_attack_rating.append(np.nanmean(rates[1][2]))

        prev_teams = teams
        rates = (([], [], []), ([], [], []))

    home_flag = (ratings.iloc[i]['home_or_away'] == 'home')
    pos = ratings.iloc[i]['position']
    r = float(ratings.iloc[i]['rating'])
    if home_flag:
        if pos in ('GK', 'DC', 'DL', 'DR', 'DML', 'DMR'):
            rates[0][0].append(r)
        if pos in ('DMC', 'MC', 'ML', 'MR', 'AMC'):
            rates[0][1].append(r)
        if pos in ('AML', 'AMR', 'FWL', 'FWR', 'FW'):
            rates[0][2].append(r)
    else:
        if pos in ('GK', 'DC', 'DL', 'DR', 'DML', 'DMR'):
            rates[1][0].append(r)
        if pos in ('DMC', 'MC', 'ML', 'MR', 'AMC'):
            rates[1][1].append(r)
        if pos in ('AML', 'AMR', 'FWL', 'FWR', 'FW'):
            rates[1][2].append(r)

home_defense_rating.append(np.nanmean(rates[0][0]))
away_defense_rating.append(np.nanmean(rates[1][0]))
home_midfield_rating.append(np.nanmean(rates[0][1]))
away_midfield_rating.append(np.nanmean(rates[1][1]))
home_attack_rating.append(np.nanmean(rates[0][2]))
away_attack_rating.append(np.nanmean(rates[1][2]))

In [21]:
len(home_defense_rating), len(home_midfield_rating), len(home_attack_rating)

(16332, 16332, 16332)

In [22]:
len(away_defense_rating), len(away_midfield_rating), len(away_attack_rating)

(16332, 16332, 16332)

In [23]:
matches['home_defense_rating'] = home_defense_rating
matches['away_defense_rating'] = away_defense_rating
matches['home_midfield_rating'] = home_midfield_rating
matches['away_midfield_rating'] = away_midfield_rating
matches['home_attack_rating'] = home_attack_rating
matches['away_attack_rating'] = away_attack_rating

In [24]:
home = 'Roma'
away = 'Inter'
season = '2022/2023'

c_1 = (matches['home_team'] == home)
c_2 = (matches['away_team'] == away)
c_3 = (matches['season'] == season)

matches[c_1 & c_2 & c_3]

,Unnamed: 0,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,datetime,home_standings,away_standings,home_points,away_points,home_points_last_4_matches,away_points_last_4_matches,home_points_last_2_home_matches,away_points_last_2_away_matches,home_goals_last_4_matches,away_goals_last_4_matches,home_goals_last_2_home_matches,away_goals_last_2_away_matches,home_goals_against_last_4_matches,away_goals_against_last_4_matches,home_goals_against_last_2_home_matches,away_goals_against_last_2_away_matches,home_mean_xg_last_4_matches,away_mean_xg_last_4_matches,home_mean_xg_last_2_home_matches,away_mean_xg_last_2_away_matches,home_mean_xg_against_last_4_matches,away_mean_xg_against_last_4_matches,home_mean_xg_against_last_2_home_matches,away_mean_xg_against_last_2_away_matches,home_formation,away_formation,home_defense_rating,away_defense_rating,home_midfield_rating,away_midfield_rating,home_attack_rating,away_attack_rating
16118,16284,Roma,Inter,May 06 2023,Serie A,2022/2023,0,2,0.88,1.22,2023-05-06,7,5,58,60,5.0,9.0,4.0,6.0,6.0,12.0,4.0,9.0,5.0,2.0,1.0,0.0,1.605,2.265,1.8,2.17,1.015,0.7775,1.25,0.645,1,3,6.145,7.316667,6.725,6.93,6.09,7.005


In [ ]:
c_1 = (ratings['home_team'] == home)
c_2 = (ratings['away_team'] == away)
c_3 = (ratings['season'] == season)

ratings[c_1 & c_2 & c_3]

Get stats for the last 4 matches (both teams), last 2 home matches (home team), last 2 away matches (away team):

1. Mean defense rating
2. Mean midfield rating
3. Mean attack rating

In [26]:
team_stats = dict()

for team in matches['home_team'].unique():
    team_stats[team] = {
        'defense': [np.nan, np.nan, np.nan, np.nan],
        'home_defense': [np.nan, np.nan],
        'away_defense': [np.nan, np.nan],
        'midfield': [np.nan, np.nan, np.nan, np.nan],
        'home_midfield': [np.nan, np.nan],
        'away_midfield': [np.nan, np.nan],
        'attack': [np.nan, np.nan, np.nan, np.nan],
        'home_attack': [np.nan, np.nan],
        'away_attack': [np.nan, np.nan]
    }

In [27]:
home_defense_rating_last_4_matches = []
away_defense_rating_last_4_matches = []
home_defense_rating_last_2_home_matches = []
away_defense_rating_last_2_away_matches = []

home_midfield_rating_last_4_matches = []
away_midfield_rating_last_4_matches = []
home_midfield_rating_last_2_home_matches = []
away_midfield_rating_last_2_away_matches = []

home_attack_rating_last_4_matches = []
away_attack_rating_last_4_matches = []
home_attack_rating_last_2_home_matches = []
away_attack_rating_last_2_away_matches = []

for match_i in range(len(matches)):
    home, away = matches.iloc[match_i]['home_team'], matches.iloc[match_i]['away_team']

    home_defense_rating = matches.iloc[match_i]['home_defense_rating']
    home_midfield_rating = matches.iloc[match_i]['home_midfield_rating']
    home_attack_rating = matches.iloc[match_i]['home_attack_rating']

    away_defense_rating = matches.iloc[match_i]['away_defense_rating']
    away_midfield_rating = matches.iloc[match_i]['away_midfield_rating']
    away_attack_rating = matches.iloc[match_i]['away_attack_rating']

    home_defense_rating_last_4_matches.append(np.mean(team_stats[home]['defense']))
    away_defense_rating_last_4_matches.append(np.mean(team_stats[away]['defense']))
    home_defense_rating_last_2_home_matches.append(np.mean(team_stats[home]['home_defense']))
    away_defense_rating_last_2_away_matches.append(np.mean(team_stats[away]['away_defense']))

    home_midfield_rating_last_4_matches.append(np.mean(team_stats[home]['midfield']))
    away_midfield_rating_last_4_matches.append(np.mean(team_stats[away]['midfield']))
    home_midfield_rating_last_2_home_matches.append(np.mean(team_stats[home]['home_midfield']))
    away_midfield_rating_last_2_away_matches.append(np.mean(team_stats[away]['away_midfield']))

    home_attack_rating_last_4_matches.append(np.mean(team_stats[home]['attack']))
    away_attack_rating_last_4_matches.append(np.mean(team_stats[away]['attack']))
    home_attack_rating_last_2_home_matches.append(np.mean(team_stats[home]['home_attack']))
    away_attack_rating_last_2_away_matches.append(np.mean(team_stats[away]['away_attack']))

    team_stats[home] = {
        'defense': [home_defense_rating, team_stats[home]['defense'][0], team_stats[home]['defense'][1], team_stats[home]['defense'][2]],
        'home_defense': [home_defense_rating, team_stats[home]['home_defense'][0]],
        'away_defense': team_stats[home]['away_defense'],
        'midfield': [home_midfield_rating, team_stats[home]['midfield'][0], team_stats[home]['midfield'][1], team_stats[home]['midfield'][2]],
        'home_midfield': [home_midfield_rating, team_stats[home]['home_midfield'][0]],
        'away_midfield': team_stats[home]['away_midfield'],
        'attack': [home_attack_rating, team_stats[home]['attack'][0], team_stats[home]['attack'][1], team_stats[home]['attack'][2]],
        'home_attack': [home_attack_rating, team_stats[home]['home_attack'][0]],
        'away_attack': team_stats[home]['away_attack']
    }

    team_stats[away] = {
        'defense': [away_defense_rating, team_stats[away]['defense'][0], team_stats[away]['defense'][1], team_stats[away]['defense'][2]],
        'away_defense': [away_defense_rating, team_stats[away]['away_defense'][0]],
        'home_defense': team_stats[away]['home_defense'],
        'midfield': [away_midfield_rating, team_stats[away]['midfield'][0], team_stats[away]['midfield'][1], team_stats[away]['midfield'][2]],
        'away_midfield': [away_midfield_rating, team_stats[away]['away_midfield'][0]],
        'home_midfield': team_stats[away]['home_midfield'],
        'attack': [away_attack_rating, team_stats[away]['attack'][0], team_stats[away]['attack'][1], team_stats[away]['attack'][2]],
        'away_attack': [away_attack_rating, team_stats[away]['away_attack'][0]],
        'home_attack': team_stats[away]['home_attack']
    }

matches['home_defense_rating_last_4_matches'] = home_defense_rating_last_4_matches
matches['away_defense_rating_last_4_matches'] = away_defense_rating_last_4_matches
matches['home_defense_rating_last_2_home_matches'] = home_defense_rating_last_2_home_matches
matches['away_defense_rating_last_2_away_matches'] = away_defense_rating_last_2_away_matches
matches['home_midfield_rating_last_4_matches'] = home_midfield_rating_last_4_matches
matches['away_midfield_rating_last_4_matches'] = away_midfield_rating_last_4_matches
matches['home_midfield_rating_last_2_home_matches'] = home_midfield_rating_last_2_home_matches
matches['away_midfield_rating_last_2_away_matches'] = away_midfield_rating_last_2_away_matches
matches['home_attack_rating_last_4_matches'] = home_attack_rating_last_4_matches
matches['away_attack_rating_last_4_matches'] = away_attack_rating_last_4_matches
matches['home_attack_rating_last_2_home_matches'] = home_attack_rating_last_2_home_matches
matches['away_attack_rating_last_2_away_matches'] = away_attack_rating_last_2_away_matches

In [28]:
home_team = 'Manchester City'
away_team = 'Liverpool'

tmp = matches[
    (matches['home_team'] == home_team)
    &
    (matches['away_team'] == away_team)
].sort_values(by=['datetime'], ascending=False).iloc[0]

tmp

Unnamed: 0                                             6069
home_team                                   Manchester City
away_team                                         Liverpool
date                                          April 01 2023
league                                                  EPL
season                                            2022/2023
home_score                                                4
away_score                                                1
home_xg                                                3.01
away_xg                                                0.32
datetime                                         2023-04-01
home_standings                                            2
away_standings                                            8
home_points                                              61
away_points                                              42
home_points_last_4_matches                             10.0
away_points_last_4_matches              

In [29]:
matches = matches.drop(columns=['Unnamed: 0'])

In [30]:
matches.head()

,home_team,away_team,date,league,season,home_score,away_score,home_xg,away_xg,datetime,home_standings,away_standings,home_points,away_points,home_points_last_4_matches,away_points_last_4_matches,home_points_last_2_home_matches,away_points_last_2_away_matches,home_goals_last_4_matches,away_goals_last_4_matches,home_goals_last_2_home_matches,away_goals_last_2_away_matches,home_goals_against_last_4_matches,away_goals_against_last_4_matches,home_goals_against_last_2_home_matches,away_goals_against_last_2_away_matches,home_mean_xg_last_4_matches,away_mean_xg_last_4_matches,home_mean_xg_last_2_home_matches,away_mean_xg_last_2_away_matches,home_mean_xg_against_last_4_matches,away_mean_xg_against_last_4_matches,home_mean_xg_against_last_2_home_matches,away_mean_xg_against_last_2_away_matches,home_formation,away_formation,home_defense_rating,away_defense_rating,home_midfield_rating,away_midfield_rating,home_attack_rating,away_attack_rating,home_defense_rating_last_4_matches,away_defense_rating_last_4_matches,home_defense_rating_last_2_home_matches,away_defense_rating_last_2_away_matches,home_midfield_rating_last_4_matches,away_midfield_rating_last_4_matches,home_midfield_rating_last_2_home_matches,away_midfield_rating_last_2_away_matches,home_attack_rating_last_4_matches,away_attack_rating_last_4_matches,home_attack_rating_last_2_home_matches,away_attack_rating_last_2_away_matches
0,Reims,Paris Saint Germain,August 08 2014,Ligue 1,2014/2015,2,2,1.37,2.66,2014-08-08,16,15,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6.6500,6.600,6.946000,7.200000,6.990000,7.28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Evian Thonon Gaillard,Caen,August 09 2014,Ligue 1,2014/2015,0,3,0.81,1.24,2014-08-09,5,4,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6.1975,7.404,6.286667,7.988000,6.350000,8.60,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Guingamp,Saint-Etienne,August 09 2014,Ligue 1,2014/2015,0,2,0.63,1.50,2014-08-09,5,18,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,0,6.1080,7.554,6.547500,7.296667,6.085000,7.32,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Lille,Metz,August 09 2014,Ligue 1,2014/2015,0,0,1.54,0.06,2014-08-09,7,11,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,7.2160,7.480,6.693333,6.776000,6.523333,5.86,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Montpellier,Bordeaux,August 09 2014,Ligue 1,2014/2015,0,1,1.02,0.75,2014-08-09,13,7,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,0,6.7280,7.240,6.796000,6.734000,6.440000,7.48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
matches.to_csv('matches_ultimate.csv')